<h1> Máquinas de aprendizaje: Tarea 1


Felipe Araya Barrera - 201173515-3  
Sebastián Vergara Miranda - 201173515-3  
9 de septiembre de 2016

<h1> Importaciones necesarias (secciones 1, 2, 3 y 4)

In [1]:
#!usr/bin/env python
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import sklearn.linear_model as lm
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn import cross_validation
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.sparse import csr_matrix
from scipy.io import mmread

<h1> 1. Regresión Lineal Ordinaria (LSS)

**1.1 Construcción de dataframe**

Se construye un dataframe con los datos provistos:

In [2]:
url = 'http://statweb.stanford.edu/~tibs/ElemStatLearn/datasets/prostate.data'
df = pd.read_csv(url, sep='\t', header=0)

Luego, se realizan los siguientes ajustes sobre este:

1. Se elimina columna sin nombre que especifica la posición de cada registro en el dataframe (línea 5). <br> <br>
  
2. Se almacena por separado la variable *train*, la cual indica si el registro pertenece o no
al conjunto de entrenamiento (línea 6). <br> <br>

3. Se crea un arreglo que realiza un cambio de notación para los valores que puede tomar
la variable *train*, de tal manera que si posee el valor 'T', se cambia por **True**, mientras que si el valor es 'F' se cambia por **False** (línea 7). <br> <br>

4. Se crea un arreglo en el que se indica si cada registro pertenece o no al conjunto de
prueba (línea 8). <br> <br>

5. Se elimina la variable train del dataframe creado (línea 9).

In [3]:
df = df.drop('Unnamed: 0', axis=1)
istrain_str = df['train']
istrain = np.asarray([True if s == 'T' else False for s in istrain_str])
istest = np.logical_not(istrain)
df = df.drop('train',axis=1)

**1.2 Descripción de dataset**

El dataset se compone de 97 registros (pacientes), cada uno de los cuáles está descrito por
9 variables, las que se detallan a continuación:

1. *lcavol* : Logaritmo del volumen de cáncer presente. <br> <br>
    
2. *lweight*: Logaritmo del peso de la próstata. <br> <br> 

3. *age*: Edad. <br> <br>  

4. *lbph*: Logaritmo de la cantidad de hiperplasia benigna de próstata. <br> <br> 

5. *svi*: Indica si existe invasión de la vesícula seminal o no. <br> <br> 

6. *lcp*: Logaritmo de la penetración capsular. <br> <br>

7. *gleason*: Medida del grado de agresividad del cáncer, en base a la escala de Gleason. <br> <br>  

8. *pgg45* : Porcentaje que representa la presencia de los patrones de Gleason 4 y 5. <br> <br>  

9. *lpsa*: Logaritmo del nivel de antígeno prostático específico (PSA). <br> <br>
Además, no existen valores nulos para ningún registro. Más información acerca del dataset (principalmente estadística) puede encontrarse al ejecutar los siguientes comandos: 

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 9 columns):
lcavol     97 non-null float64
lweight    97 non-null float64
age        97 non-null int64
lbph       97 non-null float64
svi        97 non-null int64
lcp        97 non-null float64
gleason    97 non-null int64
pgg45      97 non-null int64
lpsa       97 non-null float64
dtypes: float64(5), int64(4)
memory usage: 6.9 KB


In [5]:
df.describe()

,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa
count,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000,97.000000
mean,1.350010,3.628943,63.865979,0.100356,0.216495,-0.179366,6.752577,24.381443,2.478387
std,1.178625,0.428411,7.445117,1.450807,0.413995,1.398250,0.722134,28.204035,1.154329
min,-1.347074,2.374906,41.000000,-1.386294,0.000000,-1.386294,6.000000,0.000000,-0.430783
25%,0.512824,3.375880,60.000000,-1.386294,0.000000,-1.386294,6.000000,0.000000,1.731656
50%,1.446919,3.623007,65.000000,0.300105,0.000000,-0.798508,7.000000,15.000000,2.591516
75%,2.127041,3.876396,68.000000,1.558145,0.000000,1.178655,7.000000,40.000000,3.056357
max,3.821004,4.780383,79.000000,2.326302,1.000000,2.904165,9.000000,100.000000,5.582932


**1.3 Normalización de datos**

Antes de llevar la construcción de un modelo predictivo, es necesario normalizar los datos,
pues se está trabajando con variables medidas en unidades y escalas diferentes. Al normalizar,
es posible realizar comparaciones razonables entre ellas. Así, cada variable numérica se escala
en el rango [0,1]. Para esta instancia en particular, los valores de *lpsa* se mantienen respecto al dataset original. También, se observa que la media de cada variable tiende a 0 y la varianza es muy cercana a 1. Para la normalización, se lleva a cabo el siguiente procedimiento:

In [6]:
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
df_scaled['lpsa'] = df['lpsa']

**1.4 Construcción de modelo de regresión lineal ordinaria**

Para implementar el modelo, se ejecutan los siguientes pasos: <br> 
1. Primero, se separa el conjunto de datos en dos partes: La primera contiene solo las
variables predictivas ($C_1$), mientras que la segunda, sólo la variable a predecir ($C_2$).<br> <br> 
2. En $C_1$ , se agrega una nueva variable, que representa el intercepto que será utilizado por
cada registro para construir el modelo de regresión. <br> <br>
3. Tanto $C_1$ como $C_2$ se subdividen en un conjunto que contiene registros que pertenecen
al set de entrenamiento ($C_{11}$ y $C_{21}$) y en otro que agrupa los que pertenecen al set de prueba ($C_{21}$ y $C_{22}$ ). <br> <br>
4. Finalmente, para implementar la regresión lineal, se entrega a la función correspondiente
los conjuntos $C_{11}$ y $C_{21}$ como argumentos.

In [7]:
X = df_scaled.ix[: , :-1]
N = X.shape[0]
X.insert(X.shape[1], 'intercept', np.ones(N))
y = df_scaled['lpsa']
Xtrain = X[istrain]
ytrain = y[istrain]
Xtest = X[np.logical_not(istrain)]
ytest = y[np.logical_not(istrain)]
linreg = lm.LinearRegression(fit_intercept = False)
linreg.fit(Xtrain, ytrain)

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=1, normalize=False)

**1.5 Pesos y Z-score**

Se obtienen los pesos, errores estándar y Z-score para cada variable del
modelo elaborado.

In [8]:
#Pesos
weights = linreg.coef_
#Error estandar
SEM = np.asarray(Xtrain.std()) / np.sqrt(len(Xtrain))
#A partir de lo anterior, se calculan los Z-score de cada variable
Z_score = weights / SEM

La tabla 1 resume los valores obtenidos.

 <table style="width:100%">
 <caption>Tabla 1: Peso, SEM y Z-score de cada variable del modelo LSS.</caption>
  <tr>
    <th>**Variable**</th>
    <th>**Peso**</th>
    <th>**Error Estándar (SEM)**</th>
    <th>**Z-score**</th>
  </tr>
  <tr>
    <td>*lcavol*</td>
    <td>0.68</td>
    <td>0.13</td>
    <td>5.22</td>
  </tr>
  <tr>
    <td>*lweight*</td>
    <td>0.26</td>
    <td>0.14</td>
    <td>1.92</td>
  </tr>
  <tr>
    <td>*age*</td>
    <td>-0.14</td>
    <td>0.12</td>
    <td>-1.14</td>
  </tr>
  <tr>
    <td>*lbph*</td>
    <td>0.21</td>
    <td>0.12</td>
    <td>1.69</td>
  </tr>
  <tr>
    <td>*svi*</td>
    <td>0.30</td>
    <td>0.12</td>
    <td>2.44</td>
  </tr>
  <tr>
    <td>*lcp*</td>
    <td>-0.29</td>
    <td>0.12</td>
    <td>-2.33</td>
  </tr>
  <tr>
    <td>*gleason*</td>
    <td>-0.02</td>
    <td>0.12</td>
    <td>-0.18</td>
  </tr>
  <tr>
    <td>*pgg45*</td>
    <td>0.27</td>
    <td>0.13</td>
    <td>2.08</td>
  </tr>
</table> 

De acuerdo a estos resultados, las variables que presentan una mayor correlación con la
variable a predecir son *lcavol*, *svi* y *lcp*. Si se utiliza un nivel de significancia del 5 %, entonces existe evidencia suficiente para afirmar que la variable **NO** presenta una fuerte relación con la respuesta si Z-score ∈ [-1.668, 1.668]. Ante esto, existe evidencia suficiente para afirmar que las variables *age* y *gleason* no están relacionadas con la variable a predecir.

**1.6 Estimación de error de predicción**

Primeramente, se calcula el error real del modelo (error cuadrático medio), obteniéndose
MSE = 0.52.

In [9]:
yhat_test = linreg.predict(Xtest)
mse_test = np.mean(np.power(yhat_test - ytest, 2))
print mse_test

0.521274005508


Luego, se evalúan los resultados obtenidos por medio del método cross validation.
Se estudian dos casos:

**Caso 1**: K = 5

In [10]:
Xm = Xtrain.as_matrix()
ym = ytrain.as_matrix()
k_fold1 = cross_validation.KFold(len(Xm), 5)
mse_cv1 = 0
for k, (train, val) in enumerate(k_fold1):
  linreg = lm.LinearRegression(fit_intercept = False)
  linreg.fit(Xm[train], ym[train])
  yhat_val = linreg.predict(Xm[val])
  mse_fold = np.mean(np.power(yhat_val - ym[val], 2))
  mse_cv1 += mse_fold
mse_cv1 = mse_cv1 / 5
print mse_cv1

0.956514631616


Así, para K = 5, se obtiene un error $e_1$ = 0.96.

**Caso 2**: K = 10

In [11]:
k_fold2 = cross_validation.KFold(len(Xm), 10)
mse_cv2 = 0
for k, (train, val) in enumerate(k_fold2):
  linreg = lm.LinearRegression(fit_intercept = False)
  linreg.fit(Xm[train], ym[train])
  yhat_val = linreg.predict(Xm[val])
  mse_fold = np.mean(np.power(yhat_val - ym[val], 2))
  mse_cv2 += mse_fold
mse_cv2 = mse_cv2 / 10
print mse_cv2

0.757237472963


Es decir, con K = 10, se obtiene un error $e_2$ = 0.76. <br>

Se observa que para los casos 1 y 2, el error es considerablemente mayor respecto a MSE.
Esto quiere decir que existe un alto nivel de dependencia del modelo respecto a los datos
usados para construirlo. En otras palabras, el modelo está sobre ajustado (*overfitting*), por
lo que entregará buenas predicciones para los casos pertenecientes al dataset original, pero
no será el más apropiado si se desea predecir el valor de la variable de interés para casos de
pacientes que no se encuentran dentro del conjunto de datos de entrenamiento.

**1.7 Error de predicción por dato**

Se estudia la validez de suponer que los errores de predicción sobre los datos de entrenamiento siguen una distribución normal. Con la ejecución del siguiente código, se obtiene un gráfico que muestra el comportamiento de los errores:

In [13]:
#Se estima error de prediccion por cada dato de entrenamiento
yhat_train = linreg.predict(Xtrain)
ytrain_array = np.asarray(ytrain)
error = yhat_train - ytrain_array
#Se genera grafico de errores
stats.probplot(error, dist='norm', plot=plt)
plt.title('Siguen los errores de prediccion sobre el conjunto de entrenamiento una distribucion normal?')
plt.ylabel('Error dato de entrenamiento')
plt.show()

Se observa que el conjunto de errores puede representarse mediante una recta, idea que
se reafirma al notar que el coeficiente de correlación es $R^2$ = 0,9913. Como consecuencia de
lo anterior, y de acuerdo con el marco teórico, es correcto señalar que los errores siguen una
distribución normal.

<h1>2. Selección de atributos

**Selección vía FSS**

Se implementa *Forward Step-wise Selection* (FSS). Para agregar una variable al modelo, se utiliza como criterio el mínimo MSE.

In [32]:
def fss(x, y, names_x, k = 10000):
  mse_train = []
  p = x.shape[1]-1
  k = min(p, k)
  names_x = np.array(names_x)
  remaining = range(0, p)
  selected = [p]
  current_score = 0.0
  best_new_score = 0.0
  while [remaining] and len(selected)<=k :
    score_candidates = []
    for candidate in remaining:
      model = lm.LinearRegression(fit_intercept=False)
      indexes = selected + [candidate]
      x_train = x[:,indexes]
      predictions_train = model.fit(x_train, y).predict(x_train)
      residuals_train = predictions_train - y
      mse_candidate = np.mean(np.power(residuals_train, 2))
      score_candidates.append((mse_candidate, candidate))
    score_candidates.sort()
    score_candidates[:] = score_candidates[::-1]
    best_new_score, best_candidate = score_candidates.pop()
    remaining.remove(best_candidate)
    selected.append(best_candidate)
    print "selected = %s ..."%(names_x[best_candidate])
    print "total variables = %d, mse = %f"%(len(indexes),best_new_score)
    mse_train.append(best_new_score)
  return selected, mse_train

Luego, se aplica la función creada sobre los datos de entrenamiento:

In [33]:
names_regressors = ["Lcavol", "Lweight", "Age", "Lbph", "Svi", "Lcp", "Gleason", "Pgg45"]
print "FSS, error de entrenamiento:"
seleccionados, mse_train = fss(Xm,ym,names_regressors)

FSS, error de entrenamiento:
selected = Lcavol ...
total variables = 2, mse = 0.664606
selected = Lweight ...
total variables = 3, mse = 0.553610
selected = Svi ...
total variables = 4, mse = 0.521011
selected = Lbph ...
total variables = 5, mse = 0.489776
selected = Pgg45 ...
total variables = 6, mse = 0.478648
selected = Lcp ...
total variables = 7, mse = 0.455818
selected = Age ...
total variables = 8, mse = 0.439363
selected = Gleason ...
total variables = 9, mse = 0.439200


Al mismo tiempo, se modifica la función anterior para aplicar FSS sobre los datos de prueba.


In [30]:
Xm_test = Xtest.as_matrix()
ym_test = ytest.as_matrix()

def fss_test(x,x_t, y, y_t, selected, names_x):
  score = []
  indexes = []
  indexes.append(selected[0])
  for i in range(1, len(selected)):
    model = lm.LinearRegression(fit_intercept=False)
    indexes.append(selected[i])
    x_train = x[:,indexes]
    x_test = x_t[:,indexes]
    prediction_test = model.fit(x_train, y).predict(x_test)
    residuals_test = prediction_test - y_t
    mse_test = np.mean(np.power(residuals_test,2))
    score.append(mse_test)
  return score

mse_test = fss_test(Xm,Xm_test,ym,ym_test,seleccionados,names_regressors)

A continuación, se construye un gráfico con el error de entrenamiento y de pruebas en función de la cantidad de variables. Es pertinente señalar que FSS parte incluyendo el intercepto y luego empieza a agregar variables.

In [34]:
n_predictores = range(1, Xm.shape[1])
fig = plt.figure()
ax = plt.subplot(111)

ax.plot(n_predictores, mse_train, 'bo-', label='FSS train')
ax.plot(n_predictores, mse_test, 'ro-', label='FSS test')

ax.legend()

plt.xlabel('Cantidad de variables')
plt.ylabel('MSE')
plt.title('Step-wise Selection (FSS)')

plt.axis([0, 9, 0, 1])
plt.show()

De acuerdo al gráfico obtenido anteriormente, se observa que el error de entrenamiento decrece a medida que aumenta la cantidad de variables del modelo. Por otro lado, el error de prueba decrece al principio, pero después aumenta, lo que implica que el modelo está sobreajustado. Además, se puede ver que el error es mínimo cuando existen tres variables en el modelo. Dichas variables son: *Lcavol*, *Lweight* y *Svi*.

**Selección vía BSS**

Se implementa *Backward Step-wise Selection* (BSS). Para eliminar una variable, se utiliza mínimo MSE como criterio.

In [36]:
def bss(x, y, names_x):
  mse_train = []
  orden_drop = []
  names_x = np.array(names_x)
  remaining = range(0, x.shape[1])
  selected = range(0,x.shape[1])

  model = lm.LinearRegression(fit_intercept=False)
  predictions_train = model.fit(x, y).predict(x)
  residuals_train = predictions_train - y
  mse_train.append(np.mean(np.power(residuals_train, 2)))

  while (len(selected) != 1):
    score_candidates = []
    for candidate in remaining:
      model = lm.LinearRegression(fit_intercept=False)
      selected.remove(candidate)
      indexes = selected
      x_train = x[:,indexes]
      predictions_train = model.fit(x_train, y).predict(x_train)
      residuals_train = predictions_train - y
      mse_candidate = np.mean(np.power(residuals_train, 2))
      score_candidates.append((mse_candidate, candidate))
      selected.append(candidate)
    score_candidates.sort()
    score_candidates[:] = score_candidates[::-1]
    worst_new_score, worst_candidate = score_candidates.pop()
    remaining.remove(worst_candidate)
    selected.remove(worst_candidate)
    print "selected = %s ..."%(names_x[worst_candidate])
    print "total variables = %d, mse = %f"%(len(indexes),worst_new_score)
    mse_train.append(worst_new_score)
    orden_drop.append(worst_candidate)
  orden_drop.append(x.shape[1]-1)

  return orden_drop, mse_train

Luego, se aplica la función creada sobre los datos de entrenamiento:

In [37]:
names_regressors = ["Lcavol", "Lweight", "Age", "Lbph", "Svi", "Lcp", "Gleason", "Pgg45", "intercept"]
print "BSS, error de entrenamiento:"
seleccionados, mse_train  = bss(Xm,ym,names_regressors)

BSS, error de entrenamiento:
selected = Gleason ...
total variables = 8, mse = 0.439363
selected = Age ...
total variables = 7, mse = 0.455818
selected = Lcp ...
total variables = 6, mse = 0.478648
selected = Pgg45 ...
total variables = 5, mse = 0.489776
selected = Lbph ...
total variables = 4, mse = 0.521011
selected = Svi ...
total variables = 3, mse = 0.553610
selected = Lweight ...
total variables = 2, mse = 0.664606
selected = Lcavol ...
total variables = 1, mse = 1.437036


Luego, se modifica la función anterior para aplicar BSS sobre los datos de prueba.

In [38]:
def bss_test(x,x_t, y, y_t, selected, names_x):
  score = []
  indexes = selected[:]
  for i in selected:
    model = lm.LinearRegression(fit_intercept=False)
    x_train = x[:,indexes]
    x_test = x_t[:,indexes]
    prediction_test = model.fit(x_train, y).predict(x_test)
    residuals_test = prediction_test - y_t
    mse_test = np.mean(np.power(residuals_test,2))
    score.append(mse_test)
    indexes.remove(i)

  return score

mse_test = bss_test(Xm,Xm_test,ym,ym_test,seleccionados,names_regressors)

A continuación, se construye un gráfico con el error de entrenamiento y de pruebas de acuerdo al número de variables.

In [ ]:
n_predictores = range(0, Xm.shape[1])
fig = plt.figure()
ax = plt.subplot(111)

ax.plot(n_predictores, mse_train, 'bo-', label='BSS train')
ax.plot(n_predictores, mse_test, 'ro-', label='BSS test')

ax.legend()

plt.xlabel('Cantidad de variables')
plt.ylabel('MSE')
plt.title('Backward Step-wise Selection (BSS)')
plt.legend(loc=2)
plt.axis([0, 9, 0, 1.5])
plt.show()

Se puede ver en el gráfico construído que el error de entrenamiento es menor mientras menos variables tiene el modelo, tal y como debiese ocurrir según la teoría. Por otro lado, el error de prueba decrece al comienzo, pero luego de que la cantidad de variables ha disminuído demasiado, el error aumenta, puesto que no se cuenta con una cantidad apropiada de variables para elaborar un buen modelo. Además, el error mínimo de prueba se obtiene al eliminar cinco variables. Así, se cuenta con un modelo que posee las variables *Lcavol*, *Lweight* y *Svi*.

<h1>3.  Regularización

**a) Uso de *Ridge Regression***

En primera instancia se procede a implementar un modelo de regresión lineal mediante "Ridge Regression" o regresión sesgada. A partir de ello se construirá un gráfico donde
$
\begin{equation} 
\lambda
\end{equation}
$
variará entre $10^4$ y $10^{-1}$ y muestra los coeficientes obtenidos como función del parámetro de regularización. Todo esto en base al siguiente código:

In [18]:
#Ridge regresion
from sklearn.linear_model import Ridge
import matplotlib.pylab as plt
#X = X.drop('intercept', axis=1)
Xtrain = X[istrain]
ytrain = y[istrain]
names_regressors = ["Lcavol", "Lweight", "Age", "Lbph", "Svi", "Lcp", "Gleason", "Pgg45"]
alphas_ = np.logspace(4,-1,base=10)
coefs = []
model = Ridge(fit_intercept=True,solver="svd")
for a in alphas_:
    model.set_params(alpha=a)
    model.fit(Xtrain, ytrain)
    coefs.append(model.coef_)
ax = plt.gca()
for y_arr, label in zip(np.squeeze(coefs).T, names_regressors):
    #print alphas_.shape
    #print y_arr.shape
    plt.plot(alphas_, y_arr, label=label)
plt.legend()
ax.set_xscale('log')
ax.set_xlim(ax.get_xlim()[::-1]) # reverse axis
plt.xlabel('Lambda')
plt.ylabel('Pesos')
plt.title('Regresion sesgada o Ridge')
plt.axis('tight')
plt.legend(loc=2)
plt.show()

Se aprecia para este gráfico que los valores de 
$
\begin{equation} 
\lambda
\end{equation}
$ 
son prácticamente nulos entre $10^4$ y $10^{3}$. Por otro lado, las variables con mayor variabilidad, como $Lcavol$ y $Svi$, son beneficiadas porque la penalización de 
$
\begin{equation} 
\lambda
\end{equation}
$
en este caso las afecta menos. Además se puede notar que a medida que aumenta
$
\begin{equation} 
\lambda
\end{equation}
$ las variables son afectadas de manera suave por la penalización.

**b) Uso de Lasso Regression**

Ahora se hará la regresión lineal con el método Lasso, y de la misma forma que se hizo anteriormente, se construirá un gráfico con los λ ubicados (para este caso) en un intervalo entre $10^1$ y $10^{−2}$:


In [15]:
#Lasso regresion
from sklearn.linear_model import Lasso
import matplotlib.pylab as plt
alphas_ = np.logspace(1,-2,base=10)
coefs = []
model = Lasso(fit_intercept=True)
for a in alphas_:
    model.set_params(alpha=a)
    model.fit(Xtrain, ytrain)
    coefs.append(model.coef_)
ax = plt.gca()
for y_arr, label in zip(np.squeeze(coefs).T, names_regressors):
    #print alphas_.shape
    #print y_arr.shape
    plt.plot(alphas_, y_arr, label=label)
plt.legend()
ax.set_xscale('log')
ax.set_xlim(ax.get_xlim()[::-1]) # reverse axis
plt.xlabel('Lambda')
plt.ylabel('Pesos')
plt.title('Regresion Lasso')
plt.axis('tight')
plt.legend(loc=2)
plt.show()

Aquí se aprecia que la regularización con el método Lasso es mucho más fuerte en comparación que con el uso de la Ridge Regression. Además, la mayor variabilidad la presenta $Lcavol$, que se hace nulo con un 
$
\begin{equation} 
\lambda
\end{equation}
$
igual a 1. Por otro lado se identifica que la variable $Gleason$ tiene un peso nulo durante todo el intervalo de 
$
\begin{equation} 
\lambda
\end{equation}
$,
con lo que se concluye que su importancia a la hora de determinar el cáncer prostático es muy pequeña. En consecuencia es más conveniente utilizar el método Lasso para la regresión lineal, utilizando un parámetro de regulación óptimo para la situación.

**c) y d) Error de regularización con *Ridge Regression* y *Lasso Regression***

Los errores de entrenamiento y de pruebas como función del parámetro de regularización para *Ridge Regression* como para *Lasso Regression* se basa en los siguientes códigos:

In [26]:
#error ridge regresion
Xtest = X[np.logical_not(istrain)]
ytest = y[np.logical_not(istrain)]
alphas_ = np.logspace(2,-2,base=10)
coefs = []
model = Ridge(fit_intercept=True)
mse_test = []
mse_train = []
for a in alphas_:
    model.set_params(alpha=a)
    model.fit(Xtrain, ytrain)
    yhat_train = model.predict(Xtrain)
    yhat_test = model.predict(Xtest)
    mse_train.append(np.mean(np.power(yhat_train - ytrain, 2)))
    mse_test.append(np.mean(np.power(yhat_test - ytest, 2)))
ax = plt.gca()
ax.plot(alphas_,mse_train,label='train error ridge')
ax.plot(alphas_,mse_test,label='test error ridge')
plt.legend(loc=2)
ax.set_xscale('log')
ax.set_xlim(ax.get_xlim()[::-1])
plt.xlabel('lambda')
plt.ylabel('Error cuadratico medio')
plt.title('Regresion Ridge')
plt.show()


#error lasso regresion
alphas_ = np.logspace(0.5,-2,base=10)
coefs = []
model = Lasso(fit_intercept=True)
mse_test = []
mse_train = []
for a in alphas_:
    model.set_params(alpha=a)
    model.fit(Xtrain, ytrain)
    yhat_train = model.predict(Xtrain)
    yhat_test = model.predict(Xtest)
    mse_train.append(np.mean(np.power(yhat_train - ytrain, 2)))
    mse_test.append(np.mean(np.power(yhat_test - ytest, 2)))
ax = plt.gca()
ax.plot(alphas_,mse_train,label='train error lasso')
ax.plot(alphas_,mse_test,label='test error lasso')
plt.legend(loc=2)
ax.set_xscale('log')
ax.set_xlim(ax.get_xlim()[::-1])
plt.xlabel('lambda')
plt.ylabel('Error cuadratico medio')
plt.title('Regresion Lasso')
plt.show()

En ambos gráficos se puede identificar que ocurre el efecto de "sobreajuste" u *overfitting* del modelo. Cuando los valores de 
$
\begin{equation} 
\lambda
\end{equation}
$ 
van disminuyendo y se comienza a depender en mayor medida de las diferentes variables del estudio, va a llegar un punto donde los errores de validación comenzarán a ascender, mientras que los errores con los datos de entrenamiento seguirán descendiendo. Este punto de inflexión es llamado el mínimo error cuadrático medio, que para Ridge se calcula con un 
$
\begin{equation} 
\lambda
\end{equation}
$
cercano a 10 y para Lasso se asocia a un 
$
\begin{equation} 
\lambda
\end{equation}
$
cercano a 0,1.

**d) Validación Cruzada**

Para estimar el valor del parámetro de regularización en los métodos anteriores, se ocupará la validación cruzada, y se trabajará en base a los siguientes algoritmos:

In [28]:
def MSER(y,yhat): return np.mean(np.power(y-yhat,2))
Xm = Xtrain.as_matrix()
ym = ytrain.as_matrix()
k_fold = cross_validation.KFold(len(Xm),10)
best_cv_mse = float("inf")
model = Ridge(fit_intercept=True)
for a in alphas_:
    model.set_params(alpha=a)
    mse_list_k10 = [MSER(model.fit(Xm[train], ym[train]).predict(Xm[val]), ym[val]) for train, val in k_fold]
    if np.mean(mse_list_k10) < best_cv_mse:
        best_cv_mse = np.mean(mse_list_k10)
        best_alpha = a
#print "mejor parametro lambda ridge=%f, error cuadratico medio(CV)=%f"%(best_alpha,best_cv_mse)

def MSEL(y,yhat): return np.mean(np.power(y-yhat,2))
Xm = Xtrain.as_matrix()
ym = ytrain.as_matrix()
k_fold = cross_validation.KFold(len(Xm),10)
best_cv_mse = float("inf")
model = Lasso(fit_intercept=True)
for a in alphas_:
    model.set_params(alpha=a)
    mse_list_k10 = [MSEL(model.fit(Xm[train], ym[train]).predict(Xm[val]), ym[val]) for train, val in k_fold]
    if np.mean(mse_list_k10) < best_cv_mse:
        best_cv_mse = np.mean(mse_list_k10)
        best_alpha = a
#print "mejor parametro lambda lasso=%f, error cuadratico medio(CV)=%f"%(best_alpha,best_cv_mse)

Los valores quedan como lo indica la siguiente tabla:

<table style="width:100%">
 <caption>Tabla 2: Estimación del parámetro de regularización usando validación cruzada en Ridge y Lasso</caption>
  <tr>
    <th></th>
    <th>***Ridge Regression***</th>
    <th>***Lasso Regression***</th>
  </tr>
  <tr>
    <td>$
\begin{equation} 
\lambda
\end{equation}
$</td>
    <td>2.2229</td>
    <td>0.0100</td>
  </tr>
  <tr>
    <td>*Error cuadrático medio*</td>
    <td>0.7519</td>
    <td>0.7587</td>
  </tr>
</table> 

En los gráficos construidos para 3.c y 3.d se aprecia que los errores cuatráticos medios rondan los valores entre 0.4 y 0.5, mientras que para la *cross-validation*, los MSE se ubican entre 0.75 y 0.76. Esto puede deberse a que la cantidad de datos usando en esta tarea es muy pequeña, y donde también pudo haber influido el *overfitting* que ocurre en ambos modelos.

<h1>4.  Predicción de utilidades de películas

Para este caso se puede identificar que el tamaño de las matrices a trabajar son muy grandes, por lo mismo se usa el comando *csr_matrix* importado de *scipy.sparse* porque además de ser una matriz de con muchos elementos iguales a cero, es muy importante mantener el formato disperso de las entradas, ya que esto permite un gran ahorro de uso de la memoria principal. Además se utilizó el archivo donde están presentes los ganadores del premio **Oscar**.

A medida que aumenta la regularización, el valor del coeficiente también aumenta, hasta que llega a un punto en el que
tal valor comienza a descender. En ese modo de testeo se buscó calcular el coeficiente de determinación. En primera instancia se usó *Lasso Regression*, según lo que se obtuvo en la pregunta anterior, donde el coeficiente más alto obtenido es de 0.5847 con un 
$
\begin{equation} 
\lambda
\end{equation}
$ igual a $10^6$. Luego se decidió utilizar *Ridge Regression* donde el valor más alto obtenido para el coeficiente es de 0.4786 con un $
\begin{equation} 
\lambda
\end{equation}
$ igual a $10^6$. Para ambos casos este valor de coeficiente se obtuvo considerando el
$
\begin{equation} 
\lambda
\end{equation}
$
que lo maximiza, donde ninguno de ellos logra obtener el coeficiente esperado, de 0.75. El código que se muestra a continuación es el correspondiente al que usa la regresión *Lasso*:

In [ ]:
#datos de entrenamiento
x = csr_matrix(mmread('train.x.mm'))
y = np.loadtxt('train.y.dat')

#Se construye modelo, utilizando Lasso
maximizador = 10**6 #coeficiente que maximiza el coeficiente de determinacion
model = Lasso(fit_intercept=True)
model.set_params(alpha=maximizador)
model.fit(x, y)

#datos de prueba
x_test = csr_matrix(mmread('test.x.mm'))
y_test = np.loadtxt('test.y.dat')
coef = model.score(x_test, y_test)
print 'El coeficiente de determinacion es de:', coef